In [2]:
import os
%pwd

'e:\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\chicken_disease_classification'

In [5]:
#no need configuration


In [6]:
import tensorflow as tf

In [ ]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path
 #initilaizing the class
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_size:list
    params_batch_size:list
    
    

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,created_directories,save_json

In [9]:
class ConfigurationEvaluationManager:
    def __init__(
   
        
            self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        created_directories([self.config.artifacts_root])



    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )

        return eval_config

In [10]:
# obj=ConfigurationEvaluationManager()
# obj.get_validation_config().path_of_model

[2024-01-02 15:22:22,529:INFO:common:yaml file:config\config.yaml loadded successfully]
[2024-01-02 15:22:22,534:INFO:common:yaml file:params.yaml loadded successfully]
[2024-01-02 15:22:22,538:INFO:common:Created directory as:{path}]


'artifacts/training/model.h5'

In [11]:
from urllib.parse import urlparse


In [39]:
model=tf.keras.models.load_model("artifacts/training/model.h5")
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

        self.path="artifacts/training/model.h5"
        

    def eval_valid_generator(self):
        datagenerator_kwargs = dict(rescale=1 / 255, validation_split=0.3)

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.eval_valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        print(scores)
        save_json(path=Path("scores.json"), data=scores)

    

In [40]:

config = ConfigurationEvaluationManager()
val_config = config.get_validation_config()
evaluation = Evaluation(val_config)
evaluation.evaluation()
evaluation.save_score()




[2024-01-02 15:34:55,164:INFO:common:yaml file:config\config.yaml loadded successfully]
[2024-01-02 15:34:55,169:INFO:common:yaml file:params.yaml loadded successfully]
[2024-01-02 15:34:55,171:INFO:common:Created directory as:{path}]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 10s 1s/step - loss: 0.5763 - accuracy: 0.9052
{'loss': 0.5762996673583984, 'accuracy': 0.9051724076271057}
[2024-01-02 15:35:05,977:INFO:common:json file is saved at scores.json]
